In [1]:
import pandas as pd
import random 
import math
cost_matrix = pd.read_csv('Cost_matrix10.csv',header=None)
flow_matrix = pd.read_csv('Flow_matrix10.csv',header=None)

In [2]:
def generate_hubs(number_of_nodes,number_of_hubs):
    hubs = []
    while len(hubs)< number_of_hubs:
        random_number = random.randint(0,9)
        if random_number not in hubs:
            hubs.append(random_number)
    return hubs

In [3]:
def calculate_total_cost(cost_matrix,flow_matrix,alpha,solution):
    total_flow = (flow_matrix.sum()).sum()
    total_cost = 0
    for i in range(len(flow_matrix)):
        for j in range(len(flow_matrix)):
            collection_cost = flow_matrix[i][j] * cost_matrix[i][solution[i]]
            transportation_cost = flow_matrix[i][j] * cost_matrix[solution[i]][solution[j]] * alpha
            distribution_cost = flow_matrix[i][j] * cost_matrix[solution[j]][j]
            cost = collection_cost + transportation_cost + distribution_cost
            total_cost += cost
    return total_cost/total_flow

#  get_initial_solution

In [4]:
def get_initial_solution(cost_matrix,flow_matrix,alpha,number_of_hubs):
    solution = [0 for _ in range(len(flow_matrix))]
    #solution = []
    #randomly select certain number of hubs
    hubs = generate_hubs(len(flow_matrix),number_of_hubs)
    #hubs = [3,5,6]
    '''#allocate nodes to their nearest hubs
    for i in range(len(cost_matrix)):
        target_hub = hubs[0]
        for hub in hubs:
            if cost_matrix[i][hub]< cost_matrix[i][target_hub]:
                target_hub = hub
        solution.append(target_hub)'''
    #randomly allocate nodes to hubs
    for i in range(len(flow_matrix)):
        if i not in hubs:
            random_number = random.randint(0,number_of_hubs-1)
            solution[i] = hubs[random_number]
        else:
            solution[i] = i
    # calculate total_cost follow Spoke-Hub-Hub-spoke strategy
    whole_cost = calculate_total_cost(cost_matrix,flow_matrix,alpha,solution)
    return hubs,solution,whole_cost

# Tabu_Search 

In [5]:
def Tabu_Search(cost_matrix,flow_matrix,alpha,number_of_hubs):
    current_hubs,current_solution,whole_cost = get_initial_solution(cost_matrix,flow_matrix,alpha,number_of_hubs)
    print(current_hubs)
    best_solution = {}
    best_solution['solution'] = current_solution.copy()
    best_solution['cost'] = whole_cost
    tabulist = []
    interation = 0
    while interation < 50:
        neighbours = []
        for i in range(len(flow_matrix)):
            if i not in current_hubs and i not in tabulist:
                for hub in current_hubs:
                    if hub != current_solution[i]:
                        neighbour_solution = current_solution.copy()
                        neighbour_solution[i] = hub
                        neighbour_dict = {}
                        neighbour_dict['solution'] = neighbour_solution.copy()
                        neighbour_dict['node'] = i
                        neighbour_dict['cost'] = calculate_total_cost(cost_matrix,flow_matrix,alpha,neighbour_solution)
                        neighbours.append(neighbour_dict)
        best_neighbour = sorted(neighbours,key = (lambda neighbour: neighbour['cost']))[0].copy()
        #refresh tabu list
        if interation < 5:
            tabulist.append(best_neighbour['node'])
        else:
            tabulist[interation%5] = best_neighbour['node']
        #refresh current solution
        current_solution = best_neighbour['solution'].copy()
        #refresh best solution 
        if best_neighbour['cost'] < best_solution['cost']:
            best_solution['solution'] = best_neighbour['solution'].copy()
            best_solution['cost'] = best_neighbour['cost']
        #refresh move
        interation += 1
    return best_solution

# Simulated_Annealing

In [125]:
def Simulated_Annealing(cost_matrix,flow_matrix,alpha,number_of_hubs):
    current_hubs,current_solution,current_cost = get_initial_solution(cost_matrix,flow_matrix,alpha,number_of_hubs)
    best_solution = {}
    best_solution['solution'] = current_solution.copy()
    best_solution['cost'] = current_cost
    interation = 0
    r = 0
    theda_min = 1
    theda = 1.2
    lumbda = 0.8
    while interation < 50:
        #randomly generate a neighbour
        while 1:
            node = random.randint(0,9)
            if node not in current_hubs:
                while 1:
                    change = random.randint(0,2)
                    if current_hubs[change] != current_solution[node]:
                        break
                break
        neighbour_solution = current_solution.copy()
        neighbour_solution[node] = current_hubs[change_hub]
        neighbour_cost = calculate_total_cost(cost_matrix,flow_matrix,alpha,neighbour_solution)
        #determine whether accept a neighbour
        if neighbour_cost < current_cost:
            current_solution = neighbour_solution.copy()
            current_cost = neighbour_cost
            if current_cost < best_solution['cost']:
                best_solution['solution'] = current_solution.copy()
                best_solution['cost'] = current_cost
                r = 0
        else:
            R = random.random()
            p = math.exp((current_cost-neighbour_cost)/theda)
            if R < p:
                if current_cost != neighbour_cost:
                    r = r+1
                current_solution = neighbour_solution.copy()
                current_cost = neighbour_cost
                theda = theda_min + lumbda*math.log(1+r)
        interation +=1
    return best_solution

 # Reduced_variable_neighbourhood_search

In [143]:
import random
def neighbourhood_structure1(hubs,solution):
    while 1:
        node = random.randint(0,9)
        if node not in hubs:
            hub_index = random.randint(0,2)
            for i in range(len(solution)):
                if solution[i] == hubs[hub_index]:
                    solution[i] = node
            hubs[hub_index] = node
        break
    return hubs,solution                    

In [144]:
def neighbourhood_structure2(hubs,solution):
    while 1:
        node1 = random.randint(0,9)
        if node1 not in hubs:
            while 1:
                node2 = random.randint(0,9)
                if node2 not in hubs and node2 != node1 and solution[node1] != solution[node2]:
                    box = solution[node1]
                    solution[node1] = solution[node2]
                    solution[node2] = box
                    break
            break
    return hubs,solution

In [145]:
def neighbourhood_structure3(hubs,solution):
    while 1:
        node = random.randint(0,9)
        if node not in hubs:
            while 1:
                change_hub_index = random.randint(0,2)
                if hubs[change_hub_index] != solution[node]:
                    break
            break
    solution[node] = hubs[change_hub_index]
    return hubs,solution

In [212]:
def neighbourhood_structure4(hubs,solution):
    hub1_index = random.randint(0,2)
    while 1:
        hub2_index = random.randint(0,2)
        if hub2_index != hub1_index:
            break
    for i in range(len(solution)):
        if solution[i] == hubs[hub1_index] and solution[i] != i:
            solution[i] = hubs[hub2_index]
        elif solution[i] == hubs[hub2_index] and solution[1] != i:
            solution[i] = hubs[hub1_index]
    return hubs,solution

In [147]:
def search_neighbourhood(k,hubs,solution):
    if k == 1:
        hubs,solution = neighbourhood_structure1(hubs,solution)
    if k == 2:
        hubs,solution = neighbourhood_structure2(hubs,solution)
    if k == 3:
        hubs,solution = neighbourhood_structure3(hubs,solution)
    if k == 4:
        hubs,solution = neighbourhood_structure4(hubs,solution)
    return hubs,solution

In [160]:
def reduced_variable_neighbourhood_search(cost_matrix,flow_matrix,alpha,number_of_hubs):
    current_hubs,current_solution,current_cost = get_initial_solution(cost_matrix,flow_matrix,alpha,number_of_hubs)
    best_solution = {}
    best_solution['solution'] = current_solution.copy()
    best_solution['cost'] = current_cost
    interation = 0
    while interation < 20:
        k = 1
        #go though 4 neibourghhood structure
        while k < 5:
            neighbour_solution = current_solution.copy()
            neighbour_hubs = current_hubs.copy()
            search_neighbourhood(k,neighbour_hubs,neighbour_solution)
            neighbour_cost = calculate_total_cost(cost_matrix,flow_matrix,alpha,neighbour_solution)
            if neighbour_cost < current_cost:
                current_hubs = neighbour_hubs.copy()
                current_solution = neighbour_solution.copy()
                current_cost = neighbour_cost
                k += 1
                if neighbour_cost < best_solution['cost']:
                    best_solution['solution'] = neighbour_solution.copy()
                    best_solution['cost'] = neighbour_cost
            else:
                k += 1
        interation += 1
    return best_solution
            

In [161]:
solution1 = reduced_variable_neighbourhood_search(cost_matrix,flow_matrix,0.2,3)
solution1

{'cost': 495.67558352919735, 'solution': [5, 5, 5, 3, 3, 5, 6, 6, 5, 6]}

# Revised Reduced Variable Neighbourhood Search 

In [6]:
def get_best_neighbour_solution(solution_list,alpha):
    best_neighbour_solution = []
    best_neighbour_cost = 10000
    for item in solution_list:
        neighbour_cost = calculate_total_cost(cost_matrix,flow_matrix,alpha,item)
        if neighbour_cost < best_neighbour_cost:
            best_neighbour_solution = item.copy()
            best_neighbour_cost = neighbour_cost
    return best_neighbour_solution,best_neighbour_cost

In [7]:
def get_best_solution_in_neighbourhood_structure1(cost_matrix,flow_matrix,alpha,hubs,solution):
    nodes = []
    neighbours = []
    for item in [0,1,2,3,4,5,6,7,8,9]:
        if item not in hubs:
            nodes.append(item)
    #print(nodes)
    for hub in hubs:
        for node in nodes:
            neighbour_solution = solution.copy()
            for i in range(len(solution)):
                if neighbour_solution[i] == hub:
                    neighbour_solution[i] = node
            neighbours.append(neighbour_solution)
    #print(len(neighbours))
    best_neighbour_solution,best_neighbour_cost = get_best_neighbour_solution(neighbours,alpha)
    return best_neighbour_solution


In [8]:
def get_best_solution_in_neighbourhood_structure2(cost_matrix,flow_matrix,alpha,hubs,solution):
    nodes = []
    neighbours = []
    for item in [0,1,2,3,4,5,6,7,8,9]:
        if item not in hubs:
            nodes.append(item)
    for i in range(len(nodes)-1):
        for j in range(i+1,len(nodes)):
            neighbour_solution = solution.copy()
            box = neighbour_solution[i]
            neighbour_solution[i] = neighbour_solution[j]
            neighbour_solution[j] = box
            neighbours.append(neighbour_solution)
    #print(len(neighbours))
    best_neighbour_solution,best_neighbour_cost = get_best_neighbour_solution(neighbours,alpha)
    return best_neighbour_solution

In [9]:
def get_best_solution_in_neighbourhood_structure3(cost_matrix,flow_matrix,alpha,hubs,solution):
    nodes = []
    neighbours = []
    for item in [0,1,2,3,4,5,6,7,8,9]:
        if item not in hubs:
            nodes.append(item)
    for node in nodes:
        for hub in hubs:
            neighbour_solution = solution.copy()
            if hub != neighbour_solution[node]:
                neighbour_solution[node] = hub
                neighbours.append(neighbour_solution)
    best_neighbour_solution,best_neighbour_cost = get_best_neighbour_solution(neighbours,alpha)
    return best_neighbour_solution

In [10]:
def get_best_solution_in_neighbourhood_structure4(cost_matrix,flow_matrix,alpha,hubs,solution):
    neighbours = []
    for i in range(len(hubs)-1):
        for j in range(i+1,len(hubs)):
            neighbour_solution = solution.copy()
            for n in range(len(neighbour_solution)):
                if neighbour_solution[n] == hubs[i] and neighbour_solution[n] != n:
                    neighbour_solution[n] = hubs[j]
                elif neighbour_solution[n] == hubs[j] and neighbour_solution[n] != n:
                    neighbour_solution[n] = hubs[i]
            neighbours.append(neighbour_solution)
    #print(len(neighbours))
    best_neighbour_solution,best_neighbour_cost = get_best_neighbour_solution(neighbours,alpha)
    return best_neighbour_solution               

In [11]:
def get_best_solution_among_all_beighbourhood_structure(cost_matrix,flow_matrix,alpha,hubs,solution):
    all_best_neighbours = []
    all_best_neighbours.append(get_best_solution_in_neighbourhood_structure1(cost_matrix,flow_matrix,alpha,hubs,solution))
    all_best_neighbours.append(get_best_solution_in_neighbourhood_structure2(cost_matrix,flow_matrix,alpha,hubs,solution))
    all_best_neighbours.append(get_best_solution_in_neighbourhood_structure3(cost_matrix,flow_matrix,alpha,hubs,solution))
    all_best_neighbours.append(get_best_solution_in_neighbourhood_structure4(cost_matrix,flow_matrix,alpha,hubs,solution))
    #print(all_best_neighbours)
    best_all_neighbour_solution,best_all_neighbour_cost = get_best_neighbour_solution(all_best_neighbours,alpha)
    return best_all_neighbour_solution,best_all_neighbour_cost               

In [16]:
def RRVNS(cost_matrix,flow_matrix,alpha,number_of_hubs):
    current_hubs,current_solution,current_cost = get_initial_solution(cost_matrix,flow_matrix,alpha,number_of_hubs)
    best_solution = {}
    best_solution['solution'] = current_solution.copy()
    best_solution['cost'] = current_cost
    interation = 0
    while interation < 50:
        solution1, cost1 = get_best_solution_among_all_beighbourhood_structure(cost_matrix,flow_matrix,alpha,current_hubs,current_solution)
        new_hubs = []
        for item in solution1:
            if item not in new_hubs:
                new_hubs.append(item)
        current_solution = solution1.copy()
        current_hubs = new_hubs.copy()
        current_cost = cost1
        if cost1 < best_solution['cost']:
            best_solution['solution'] = solution1
            best_solution['cost'] = cost1
        interation += 1
    return best_solution

In [17]:
RRVNS(cost_matrix,flow_matrix,0.2,3)

{'cost': 491.9343312144026, 'solution': [5, 5, 5, 3, 5, 5, 6, 6, 5, 6]}